# EECE 5554 Lab 3: Introduction to the VectorNav Inertial Measurement Unit (IMU)

Lab 3 introduces the VectorNav IMU and will guide you through how to read from the IMU and write a ROS publisher for IMU data.

### Goals and Objectives
We would like to understand sources of noise in sensors and in the surrounding environment, and how to characterize and choose IMU sensors for different robotic applications. We will write a device driver for the IMU, collect rotation, acceleration, magnetometer, and rotational rate data, and analyze noise parameters using Allan variance plots. 

### By the end of this lab, we expect that a well-prepared student will be able to:
- Write a device driver that communicates with the VectorNav IMU over USB serial 
- Identify device parameters (e.g., angle random walk, bias stability, rate random walk) and sources of error from Allan variance data 

### Collaboration Policy for this Lab
- You may confer with classmates to get general help on approach for writing ROS nodes or analysis scripts. You should not wholesale copy multiple lines of code from classmates, any other students' assignments, or from LLMs such as GPT3.

### Individual Work: 
- Everyone needs to write their own device driver for the IMU.
- Everyone needs to collect stationary data. 
- Analysis on the collected dataset should be done individually. 
  
### Group Work:
- Please share the IMUs amongst your group.
  
## Information of Note

### Hardware: 
- Vectornav VN-100 IMU 

### Software:
- The python package Allan Tools. Please see the Allan Tools notebook for links and details.

### The IMU driver:
You will write a driver that:
- Merges sensor readings from the IMU to standard messages
- Publishes the message

### Data sets: 
- stationary data set for 10 minutes of data collected when the IMU is far away from any sources of vibrations or electrical noise.
- one 5 h rosbag, supplied by the course and posted to Piazza.

### Repository structure:

You will need to follow the naming convention laid out here to have an appropriate structure that can be checked by the Lab 3 autograder and to receive credit for your work. Please make sure you are following these conventions **exactly**-- spaces and capitalization matter!

Your Github repo should look like the table below. **Please note that your build and devel directories should not be pushed to Git!**

| EECE5554 | | | | | |
| :----- | :----- | :----- | :----- | :----- | :----- |
| | imu/ | | | | |
| | | src/ | | | | 
| | | | vn_driver/ | | |
| | | | | CMakeLists.txt | |
| | | | | package.xml | |
| | | | | python/ | | 
| | | | | | driver.py | 
| | | | | msg/ | | 
| | | | | | Vectornav.msg | 
| | | | | launch/ | | 
| | | | | | driver.launch |
| | | analysis/ | | | |
| | | | your analysis scripts | | |
| | | data/ | | | |
| | | | your .bag files | | |


### What, where, and how to submit: 

- Push your ROS code, analysis files, and ROS bags to your Lab 3 Github repo (structure above).
- Use the Lab 3 “quiz” on Canvas to upload your other work. 

### Checklist For a Correct ROS Package Structure  
- The ROS package name is vn_driver 
- The driver is in a folder called ``python`` with the name driver.py  
- The message file name is Vectornav.msg with the names of message variables & types described in the section for writing a device driver.  
- Your topic name is “/imu”  
- You use a function “convert_to_quaternion” that converts Euler angles to Quaternions and the code for conversion is implemented by you and not using an existing library. 
- We can run your driver and service with the command ``` roslaunch vn_driver driver.launch port:=”/dev/tty*" ``` Where ```*``` can be any port  

## Onto the Lab!


## Individual work: write a device driver for IMU and do stationary noise analysis 

### Step 1: Similarly to Lab 1, use the sensor emulator with IMU data to get your driver figured out, then confirm it works with the IMU. Here’s what you’ll want your Python script to do: 
- Open a serial port with 115200 baudrate and read in the VNYMR string. 
- Parse the $VNYMR strings to get accel x, y, z, gyro x, y, z, orientation roll, pitch, yaw, and magnetometer x, y, z data. Refer to the user_manual for more information. Do not use the pynmea library. 

### Step 2: Create a function named convert_to_quaternion that is called with the Euler angles and returns the quaternion conversion. 
Please code this function yourself rather than using libraries. While you might want to use a library that is well-tested in future projects, we want to make sure you understand the process for doing this yourself in this class. 

#### Big Lab Hint: Make sure you look up what units the VectorNav is using and whether they match ROS conventions 

### Step 3: Make a custom message called “Vectornav.msg” that contains the following message types:  
- Header with the name of header 
- frame_id will be “imu1_frame”.  
- You may use the system time for the Header time stamp, but keep the highest precision by using both the secs and nsecs 
- ROS sensor_msgs/IMU with the name imu (see sensor_msgs/Imu) 
- ROS sensor_msgs/MagneticField with the name mag_field (see sensor_msgs/MagneticField) 
- A string with any name containing the raw IMU string (this can help if you find a mistake in your driver after data collection). 
     
### Step 4: Configure the vectornav to output data at 40 Hz by writing bytes to the register (UTF-8) in your Python script. 
This is a non-trivial step and should not be solved by just sleeping your driver to collect at 40 Hz. You will need to write to a register of the VectorNav.  

Before you get the actual IMU, you can check if your string is correct by writing the register bytes as an option -V to the emulator and then the write register bytes. Confirm that your rate is correct using minicom. 

### Step 5: Test everything on the actual IMU (please see hardware setup below) 

Begin a rosbag and collect at least 10 min of data with the instrument stationary and as far away as possible from your computer or other computers, moving objects, etc.  

### Step 6: Plot time series data

Please plot vs. time (starting at t = 0): 
- Rotational rate from the gyro in degrees/s on axes x, y, z (Fig 0). You can plot with different marker types or three subplots.
- Acceleration from the accelerometer in m/$s^2$ on axes x, y, z (Fig 1). You can plot with different marker types or on three subplits.
- Rotation from the VN estimation in degrees on axes x, y, z (Fig 2). You can plot with different marker types or three subplots.

And:
- Three 1D histograms of rotation in x, y, z (Fig 3).

Please convert the quaternions back to Euler angles for your analysis! 


 

## Individual work: hardware setup  

Standard Ubuntu is not designed for robotic application, so for sensors with a high sampling rate, it may not behave as we want it to. There may be latency in terms of its response to the sensor. There are a set of files under /etc/udev that dictate how Ubuntu will react to certain sensors. These files instruct Linux what to do when a particular device is plugged in. More here 

To make the VN-100 plug & play and to address Ubuntu USB latency issues: 
### Step 1: Plug in or emulate your IMU and in the terminal: ```$ dmesg | tail -20 ```

You will see your sensor attributes including product id and vendor id. These will be listed in the most recent logs after dmesg once you plug the device in.    

 
### Step 2: Use a text editor to see the contents of /etc/udev/rules.d/50-VN-100.rules. If the values in your file differ from the values below, change them accordingly and if the file doesn’t exist, 

```$ sudo nano /etc/udev/rules.d/50-VN-100.rules``` and add the contents 

```KERNEL=="ttyUSB[0-9]*", ACTION=="add", ATTRS{idVendor}=="1d6b", ATTRS{idProduct}=="0002", MODE="0666", GROUP="dialout" ```

### Step 4: Ubuntu's default latency of 16 ms cause issues with high-rate sensors. 

This problem can be solved by adding the file in ```/etc/udev/rules.d called 49-USB-LATENCY.rules``` and add the contents: 

```ACTION=="add", SUBSYSTEM=="usb-serial", DRIVER=="ftdi_sio", ATTR{latency_timer}="1" ```

### Step 5: Once the rules have been added, to get udev to recognize the rule, run the following command: 

```sudo udevadm control --reload-rules && sudo service udev restart && sudo udevadm trigger ```

### Step 6: Finally, unplug and replug the VN-100 to have it work with the new rules. 

You can verify the applied settings with below command; it should report 1 on success. Please verify each time you collect data to be safe! 

```$ cat /sys/bus/usb-serial/devices/<ttyUSB0 or your device path>/latency_timer ```

## Individual work: Allan Variance analysis  

Four .bag files have been uploaded to Piazza, which contains 5 h of stationary VN data. Your task is to make Allan variance plots of these data. Please note that these .bag files are the direct string read out from the Vectornav. Sometimes, it will give you a bad string-- you need to determine how to appropriately handle it!

### Step 1: Read in data from your bag. 

You have been assigned a letter (A, B, C, D) that corresponds to a ROSbag with IMU string data only. The VNYMR string was published to topic /vectornav. Read in the strings from the bag and perform analysis below. 

 
### Step 2: Make an Allan Variance plot. 

Use the process described in the Allan variance Jupyter notebook to make Allan variance plots for gyro x, y, z (Fig 4) and extract rate random walk (K), angle random walk (N), and bias stability (B) parameters from your data set and plots for each axis for your gyro.



### Lab Scoring

The lab will be scored according to the information below:

| Driver | 50 pct | Points | 
| :---- | :---- | :---- |
| | Correct ROS message structure | 10 | 
| | Set sample rate correctly | 10 |
| | Parse VNYMR string correctly | 10 |
| | Correct units | 10 |
| | Convert to quaternion | 10 | 
| Canvas quiz | 50 pct | | 
| | Fig. 0  | 10 | 
| | Fig. 1  | 10 | 
| | Fig. 2  | 10 |
| | Fig. 3  | 10 |
| | Fig. 4 | 10 |



